## Ejercicio - Regresión Viviendas Barcelona

- Utilizando el dataset de publicaciones de venta de viviendas en Barcelona _**viviendas_barcelona.parquet**_, entrena un modelo de regresión lineal múltiple que sea capaz de calcular el precio de venta:

    - Antes de crear el modelo y entrenarlo, haz un **EDA** de los datos.
        - ¿Qué columnas pueden ser las más importantes?
        - ¿Como se distribuyen los precios?
        - ¿Qué columnas sería factible limpiar?
        - ¿Qué significan los **np.nan**?
     
    - Cuando entiendas bien los datos con los que estás tratando, puedes proceder a realizar un preprocesamiento:
        - Elimina duplicados.
        - Elimina columnas innecesarias
        - Trata los valores perdidos **np.nan**.
        - Maneja los outliers como consideres.
        - Realiza transformaciones a los datos que consideres necesarias.
        - Codifica los datos categóricos para que el modelo los entienda (pásalos a números).
        - Considera normalizar los datos antes de utilizarlos para el entreno.
        
    - Entrena el modelo con un subconjunto de 70-80% del dataset.
      
    - Utiliza los datos restantes para comprobar cómo de bueno es el modelo:
        - Calcula **R2**, **MAE** y **MSE**.
        
- Importa las librerías que vayas a necesitar.

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Modelo
from sklearn.linear_model import LinearRegression

# Herramientas de preprocesamiento
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, TargetEncoder, OneHotEncoder

# Split
from sklearn.model_selection import train_test_split

# Métricas
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

df = pd.read_parquet("../Data/viviendas_barcelona.parquet")

In [2]:
df.shape

(27479, 25)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27479 entries, 0 to 27478
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   price                  26969 non-null  float64
 1   title                  27129 non-null  object 
 2   lat                    26873 non-null  float64
 3   lng                    26873 non-null  float64
 4   agency                 27129 non-null  object 
 5   Superficie construida  26978 non-null  object 
 6   Emisiones              16335 non-null  object 
 7   Clasificación          27129 non-null  object 
 8   Baños                  26185 non-null  object 
 9   Consumo                2132 non-null   object 
 10  Habitaciones           26171 non-null  object 
 11  Antigüedad             8724 non-null   object 
 12  Garaje                 8534 non-null   object 
 13  Terraza                12768 non-null  object 
 14  Se aceptan mascotas    1511 non-null   object 
 15  As

In [4]:
df.describe(include='all').T
# Vemos que el parametro y sera la columna price y el X el resto 

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
price,26969.0,NaN,NaN,NaN,425521.191071,569359.251647,15500.0,178000.0,270500.0,450000.0,19000000.0
title,27129,12706,Piso en venta en Centre,483,NaN,NaN,NaN,NaN,NaN,NaN,NaN
province,27479,1,barcelona,27479,NaN,NaN,NaN,NaN,NaN,NaN,NaN
location,26873,8467,Castellar del Vallès,292,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lat,26873.0,NaN,NaN,NaN,41.501189,0.299589,0.0,41.387962,41.494441,41.576186,42.283257
lng,26873.0,NaN,NaN,NaN,2.122537,0.223253,0.0,2.018452,2.130845,2.218178,3.81478
agency,27129,1329,Inmuebles de Engel & Völkers Barcelona,2187,NaN,NaN,NaN,NaN,NaN,NaN,NaN
updated,27129.0,NaN,NaN,NaN,1673576703.102952,3085560.077187,1579389000.0,1673392200.0,1673996700.0,1674169800.0,1675120080.0
timestamp,27479.0,NaN,NaN,NaN,1547226005864556.5,458334075217092.875,169789023054.0,1697790317799740.0,1697840050993678.0,1697888434903166.0,1697946906943147.0
id_1,27479,27479,34241677924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.columns

Index(['price', 'title', 'province', 'location', 'lat', 'lng', 'agency',
       'updated', 'timestamp', 'id_1', 'id_2', 'Superficie construida',
       'Emisiones', 'Clasificación', 'Baños', 'Consumo', 'Habitaciones',
       'Referencia', 'Antigüedad', 'Garaje', 'Terraza', 'Se aceptan mascotas',
       'Ascensor', 'Tipo de casa', 'Superficie útil'],
      dtype='object')

In [20]:
# Posibles columnas a descartar
# 'id_1', 'id_2', 'updated', 'timestamp', 'province', 'location', 'Referencia'
df[['id_1', 'id_2', 'updated', 'timestamp', 'province', 'location', 'Referencia']]

,id_1,id_2,updated,timestamp,province,location,Referencia
0,34241677924,104500,1.672874e+09,1.697761e+15,barcelona,"Avenida Casetes, nº 33. Centre Vila (Vilafranc...",ASR1-SRB0000036956
1,23357117200,101800,1.672701e+09,1.697761e+15,barcelona,Castellet i la Gornal,IF76306-I-00VJM3-W-02OAMJ
2,32584643160,100500,1.674861e+09,1.697761e+14,barcelona,Sant Sadurní d'Anoia,SA3541-REF_11510
3,19202422630,401700,1.673997e+09,1.697761e+15,barcelona,Carrer de Sant Pere Molanta. Olèrdola,4017-4751
4,35859429926,100500,1.674256e+09,1.697761e+15,barcelona,"Calle Carrer Serral Llarg, nº 107. Olesa de Bo...",SA3007-08353
...,...,...,...,...,...,...,...
27474,38334918184,101800,1.673997e+09,1.697947e+15,barcelona,Parets del Vallès,IF76306-I-00XK71-W-02SIMT
27475,34237375728,109300,1.673997e+09,1.697947e+15,barcelona,"Calle Ps Catalunya De, nº 2. Aiguafreda",EP831-827692
27476,37515155048,101800,1.673651e+09,1.697947e+15,barcelona,Corró d'Avall (Les Franqueses del Vallès),IF76306-I-00WBX7-W-02QFIO
27477,37581966089,170800,1.674083e+09,1.697947e+15,barcelona,Caldes de Montbui,1708-000008


In [2]:
df.drop(['id_1', 'id_2', 'updated', 'timestamp', 'province', 'location', 'Referencia'], axis=1, inplace=True)

In [4]:
print(df.head())

      price                                              title        lat  \
0   52000.0            Piso en venta en Avenida Casetes, nº 33  41.348350   
1  315000.0    Casa adosada en venta en Vilafranca del Penedes  41.276503   
2  650000.0  Casa unifamiliar en venta en Sant Sadurní d'Anoia  41.426682   
3  315000.0       Casa en venta en Carrer de Sant Pere Molanta  41.318658   
4  209900.0  Chalet en venta en Calle Carrer Serral Llarg, ...  41.369700   

        lng                                  agency Superficie construida  \
0  1.701970       Inmuebles de Aliseda Inmobiliaria                 88 m²   
1  1.652229  Inmuebles de Engel & Völkers Barcelona                179 m²   
2  1.792630               Inmuebles de SAFTI ESPAÑA                253 m²   
3  1.754891        Inmuebles de ALARCON PROPIEDADES                198 m²   
4  1.876770              Inmuebles de FINCAS MARINA                494 m²   

                           Emisiones             Clasificación Baños  \
0 

### EDA
* Revisar columnas y quitar nulos, huecos vacios , etc y convertir a numericos para 
poder utilizarlos en los metodos de prediccion

* Columnas superficies

In [10]:
# Vemos columnas Superficie
df[['Superficie construida', 'Superficie útil']]

,Superficie construida,Superficie útil
0,88 m²,None
1,179 m²,None
2,253 m²,220 m²
3,198 m²,100 m²
4,494 m²,398 m²
...,...,...
27474,181 m²,None
27475,187 m²,170 m²
27476,530 m²,None
27477,122 m²,111 m²


In [21]:
# creamos metodo para extraer la parte numerica de la columna
# superficie 1.250m2 --> 1250
def extraer_superficie_num(superficie):
    try:
        return int(superficie.split()[0].replace('.',''))
    except:
        return np.nan


# El metodo se va a ejecutar en cada celda para extraer el numero y si no hay ponen NAN
df['Superficie construida'] = df['Superficie construida'].apply(extraer_superficie_num)
median = df['Superficie construida'].median()
df['Superficie construida'] = df['Superficie construida'].fillna(median).round(0).astype(int)

In [27]:
# Convertir a numérico la columna Superficie útil y rellenar sus nulos
df['Superficie útil'] = df['Superficie útil'].apply(extraer_superficie_num)
def fill_superficie_util(fila):
    if pd.isnull(fila['Superficie útil']):
        return fila['Superficie construida'] * 0.85 # Asumimos que la superficie útil es un 15 % menos que la superficie total
    else:
        return fila['Superficie útil']
df['Superficie útil'] = df.apply(fill_superficie_util, axis=1)

* Columna emisiones

In [28]:
df['Emisiones'].unique

<bound method Series.unique of 0                            A B C D E F G
1                                     None
2          63 Kg CO2/m2 año  A B C D E F G
3          72 Kg CO2/m2 año  A B C D E F G
4                            A B C D E F G
                       ...                
27474                                 None
27475                        A B C D E F G
27476                                 None
27477                                 None
27478      19 Kg CO2/m2 año  A B C D E F G
Name: Emisiones, Length: 27479, dtype: object>

In [32]:
def extraer_emisiones_num(emision):
    try:
        return float(emision.strip().split()[0].replace(',','.')) # cambia la coma por punto y se salt alos espacios
    except:
        return np.nan # Si no hay numero lo convierte a NAN
df['Emisiones'] = df['Emisiones'].apply(extraer_emisiones_num)
# df['Emisiones'].apply(extraer_emisiones_num).isna().sum()

13899

* Columna agencia

In [33]:
df['agency'].unique()

array(['Inmuebles de Aliseda Inmobiliaria',
       'Inmuebles de Engel & Völkers Barcelona',
       'Inmuebles de SAFTI ESPAÑA', ..., 'ARTURO C', 'Propietario',
       'Alicia'], dtype=object)

In [ ]:
# Estrategia 1
# Creamos columna agency_size que represente el numeor de propiedades que tienen esa agency
#df['agency'].isna().sum()
df['agency'] = df['agency'].fillna('Other') # Rellenamos con la palabra 'other'
df['agency_size'] = df['agency'].map(df['agency'].value_counts())


# Estrategia 2
# Discretizarla en grupos: propietario individual, agencia pequeña, agencia mediana, agencia grande

# Estrategiua 3
# Crear columna con el precentil de tamaño de la agencia en base al numero de propiedades en venta

# Estrategia 4
# Precio medio de las casas de la agencia

In [37]:
pd.reset_option('display.max_rows')
df[['agency', 'agency_size']]

,agency,agency_size
0,Inmuebles de Aliseda Inmobiliaria,493
1,Inmuebles de Engel & Völkers Barcelona,2187
2,Inmuebles de SAFTI ESPAÑA,95
3,Inmuebles de ALARCON PROPIEDADES,30
4,Inmuebles de FINCAS MARINA,31
...,...,...
27474,Inmuebles de Engel & Völkers Barcelona,2187
27475,Inmuebles de ESPAI HABITAT,28
27476,Inmuebles de Engel & Völkers Barcelona,2187
27477,Inmuebles de AGC FINQUES,17


Columna ascensor

In [38]:
# ascensor_cat: con ascensor, sin ascensor, otros
df['Ascensor'].unique()

def categorizar_ascensor(valor):
    if pd.isnull(valor):
        return 0
    
    valor = valor.strip().lower()
    mapa_ascensor = {
        None: 0,
        'ascensor': 1,
        'con ascensor': 1,
        'del pis al pk': 1,
        '2': 1,
        'comedor y habitación': 1,
        '2 ascensores en finca': 1,
        'en proceso': 1,
        'no': 0,
        'ascensor disponible para acceder al terrado a tend': 1,
        '2 ascensores': 1,
        '4 personas 300kgs': 1,
        'comunica con el parking': 1,
        'en perfecto estado': 1,
        'solo para los que viven en la 3 planta.': 1,
        '1 o 2 personas': 1,
        'amplio para silla de ruedas': 1,
        'entre rellano': 1,
        '4 personas 320kgs': 1,
        '3 ( hay 3 escaleras)': 1,
        'si amplio': 1,
        'si, escalera b': 1,
        '2 ascensores ( escalera a)': 1,
        '3 personas 300kgs': 1,
        '4 ascensores': 1,
        '4': 1,
        'dos ascensores': 1,
        '3 ascensores en la finca': 1,
        'en proyecto': 1,
        'la caja de gero termo-acustico para reducir ruidos': 1,
        'emplio': 1,
        'pre-instalación': 1,
        'si': 1,
        'privado sólo para los áticos.': 0,
        'conecta parking, viviendas y trasteros': 1,
        'por uno cada siete vecinos': 1,
        'de uso exclusivo para los áticos': 0,
        'finca con ascensor': 1,
        'interior': 1,
        'no, solo es un primero': 0,
        'no hay ascensor': 0,
        'dos': 1,
        '4 por rellano': 1,
        'dos por planta': 1,
        'de 4 por planta': 1,
        'proyecto de ascensor aprobado': 1,
        'true': 1,
        'hasta el parquing': 1,
        'ascensor renovado moderno': 1,
        'nuevo': 1,
        'al ser planta baja, no hace uso de él.': 0,
        'solo 3 vecinos': 0,
        'directo a la vivienda': 1,
        '2 ascensores en la finca': 1,
        'reformado': 1
    }
    return mapa_ascensor.get(valor, 0)
    
    
df['Ascensor'] = df['Ascensor'].apply(categorizar_ascensor).astype(np.int8)

In [40]:
df.head()

,price,title,province,location,lat,lng,agency,updated,timestamp,id_1,...,Habitaciones,Referencia,Antigüedad,Garaje,Terraza,Se aceptan mascotas,Ascensor,Tipo de casa,Superficie útil,agency_size
0,52000.0,"Piso en venta en Avenida Casetes, nº 33",barcelona,"Avenida Casetes, nº 33. Centre Vila (Vilafranc...",41.348350,1.701970,Inmuebles de Aliseda Inmobiliaria,1.672874e+09,1.697761e+15,34241677924,...,3,ASR1-SRB0000036956,None,None,None,None,0,None,74.80,493
1,315000.0,Casa adosada en venta en Vilafranca del Penedes,barcelona,Castellet i la Gornal,41.276503,1.652229,Inmuebles de Engel & Völkers Barcelona,1.672701e+09,1.697761e+15,23357117200,...,4,IF76306-I-00VJM3-W-02OAMJ,None,1,Terraza,None,0,Adosada,152.15,2187
2,650000.0,Casa unifamiliar en venta en Sant Sadurní d'Anoia,barcelona,Sant Sadurní d'Anoia,41.426682,1.792630,Inmuebles de SAFTI ESPAÑA,1.674861e+09,1.697761e+14,32584643160,...,5,SA3541-REF_11510,None,2,Terraza,None,0,Unifamiliar,215.05,95
3,315000.0,Casa en venta en Carrer de Sant Pere Molanta,barcelona,Carrer de Sant Pere Molanta. Olèrdola,41.318658,1.754891,Inmuebles de ALARCON PROPIEDADES,1.673997e+09,1.697761e+15,19202422630,...,3,4017-4751,Entre 30 y 50 años,1,Terraza,Se aceptan mascotas,0,None,168.30,30
4,209900.0,"Chalet en venta en Calle Carrer Serral Llarg, ...",barcelona,"Calle Carrer Serral Llarg, nº 107. Olesa de Bo...",41.369700,1.876770,Inmuebles de FINCAS MARINA,1.674256e+09,1.697761e+15,35859429926,...,3,SA3007-08353,None,None,Terraza,None,0,None,419.90,31


In [41]:
def binarizar_terraza(valor):
    try:
        if pd.isnull(valor) or valor is None or len(valor) == 0 or 'no' in valor.strip().lower():
            return False
        else:
            return True
    except:
        return False
df['Terraza'] = df['Terraza'].apply(binarizar_terraza)

In [42]:
df['Terraza'].value_counts()

Terraza
False    14886
True     12593
Name: count, dtype: int64

Columna baño

In [ ]:
def categorizar_bathroom(valor):
    if pd.isnull(valor):
        return 0
    
    valor = int(valor)
    
    if valor <= 0:
        return 0
    elif valor >= 9:
        return 9
    else: 
        return valor
    
df['Baños'] = df['Baños'].apply(categorizar_bathroom)

Columna habitaciones

In [ ]:
def categorizar_habitaciones(valor):
    if pd.isnull(valor):
        return 0
    
    valor = int(valor)
    
    if valor <= 0:
        return 0
    elif valor >= 11:
        return 11
    else: 
        return valor

df['Habitaciones'].apply(categorizar_habitaciones).value_counts()

In [ ]:
# quitamos los que tienen precio null
df_to_pred = df[df['price'].isna()] # guardar filas con price nan por si se quieren predecir a futuro
df = df[~df['price'].isna()]

In [ ]:
df.columns

In [ ]:
df_to_eda = df[[
    'Habitaciones', 'Baños', 'Terraza', 'Ascensor', 
    'lat', 'lng', 'agency_size', 'Superficie construida', 'Superficie útil',
    'Emisiones', 'price'
]]

In [ ]:
df_to_eda = df_to_eda.dropna()

In [ ]:
# heatmap de corr
plt.figure(figsize=(10, 8))
sns.heatmap(df_to_eda.corr().round(2), annot=True, cmap='viridis')

In [ ]:
df_to_model = df[[
    'Habitaciones', 'Baños', 'Terraza',
    'agency_size', 'price'
]]

In [ ]:
X = df_to_model.drop('price', axis=1)
y = df_to_model['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
df_resultados = pd.DataFrame(columns=['Modelo', 'R2', 'MAE', 'RMSE', 'MAPE'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, root_mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor


models = {
    'RLM': LinearRegression(),
    'KNN k = 3': KNeighborsRegressor(n_neighbors=3),
    'KNN k = 5': KNeighborsRegressor(n_neighbors=5),
    'KNN k = 7': KNeighborsRegressor(n_neighbors=7),
    'KNN k = 9': KNeighborsRegressor(n_neighbors=9),
    'DT depth = 5': DecisionTreeRegressor(max_depth=5),
    'DT depth = no': DecisionTreeRegressor(),
    'RF 100': RandomForestRegressor(),
    'RF 150': RandomForestRegressor(n_estimators=150),
    'SVR 1': SVR(C=1),
    'SVR 10': SVR(C=10),
    'SVR 1000': SVR(C=1000)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) # (multiplicar mentalmente por 100)

    df_resultados.loc[len(df_resultados)] = [name, r2, mae, rmse, mape]

In [ ]:
df_resultados.sort_values('R2', ascending=False)

Necesitaríamos mejorar los datos para poder mejorar el modelado. Actualmente hay pocas correlaciones con el price.